In [1]:
import numpy as np
import numba as nb
import time
from typing import List, Tuple

# Wheel-30-Konstanten (eliminiert Vielfache von 2,3,5)
WHEEL30_SIZE = 30
WHEEL30_COPRIME = np.array([1, 7, 11, 13, 17, 19, 23, 29], dtype=np.int32)
print(f"Wheel-30 eliminiert {(30-len(WHEEL30_COPRIME))/30*100:.1f}% der Kandidaten")
print(f"Coprime Residuen mod 30: {WHEEL30_COPRIME}")

Wheel-30 eliminiert 73.3% der Kandidaten
Coprime Residuen mod 30: [ 1  7 11 13 17 19 23 29]


In [2]:
def wheel30_sieve_python(limit: int) -> List[int]:
    """Naiver Wheel-30-Generator in Python."""
    candidates = []
    k = 0
    while True:
        base = k * WHEEL30_SIZE
        if base >= limit:
            break
        for r in WHEEL30_COPRIME:
            candidate = base + r
            if candidate >= limit:
                break
            if candidate > 1:  # Skip 1
                candidates.append(candidate)
        k += 1
    return candidates


# Test
test_limit = 100
candidates = wheel30_sieve_python(test_limit)
print(f"Erste 20 Wheel-30-Kandidaten bis {test_limit}:")
print(candidates[:20])

Erste 20 Wheel-30-Kandidaten bis 100:
[np.int32(7), np.int32(11), np.int32(13), np.int32(17), np.int32(19), np.int32(23), np.int32(29), np.int32(31), np.int32(37), np.int32(41), np.int32(43), np.int32(47), np.int32(49), np.int32(53), np.int32(59), np.int32(61), np.int32(67), np.int32(71), np.int32(73), np.int32(77)]


In [3]:
@nb.njit(cache=True)
def wheel30_sieve_numba(limit: int) -> np.ndarray:
    """Wheel-30-Sieb mit Numba-JIT."""
    # Schätze maximale Anzahl der Kandidaten
    max_candidates = int(limit * 0.27)  # ~26.7% der Zahlen sind coprime zu 30
    candidates = np.empty(max_candidates, dtype=np.int32)

    idx = 0
    k = 0
    while True:
        base = k * WHEEL30_SIZE
        if base >= limit:
            break
        for i in range(len(WHEEL30_COPRIME)):
            candidate = base + WHEEL30_COPRIME[i]
            if candidate >= limit:
                break
            if candidate > 1:
                candidates[idx] = candidate
                idx += 1
        k += 1

    return candidates[:idx]


# Warmup und Test
_ = wheel30_sieve_numba(100)
candidates_nb = wheel30_sieve_numba(test_limit)
print(f"Numba-Version identisch: {np.array_equal(candidates, candidates_nb)}")

Numba-Version identisch: True


In [4]:
def brute_force_candidates(limit: int) -> List[int]:
    """Brute-Force: alle ungeraden Zahlen außer Vielfachen von 3,5."""
    return [n for n in range(3, limit, 2) if n % 3 != 0 and n % 5 != 0]


BENCHMARK_LIMIT = 1_000_000

# Benchmark Python-Varianten
start = time.perf_counter()
brute_candidates = brute_force_candidates(BENCHMARK_LIMIT)
brute_time = time.perf_counter() - start

start = time.perf_counter()
wheel_candidates = wheel30_sieve_python(BENCHMARK_LIMIT)
wheel_time = time.perf_counter() - start

# Benchmark Numba
start = time.perf_counter()
wheel_nb_candidates = wheel30_sieve_numba(BENCHMARK_LIMIT)
wheel_nb_time = time.perf_counter() - start

print(f"Kandidaten bis {BENCHMARK_LIMIT:,}:")
print(f"Brute Force:  {len(brute_candidates):,} kandidaten in {brute_time:.3f}s")
print(f"Wheel-30 Py:  {len(wheel_candidates):,} kandidaten in {wheel_time:.3f}s")
print(f"Wheel-30 NB:  {len(wheel_nb_candidates):,} kandidaten in {wheel_nb_time:.3f}s")
print(f"Speedup Numba: {wheel_time/wheel_nb_time:.1f}×")

Kandidaten bis 1,000,000:
Brute Force:  266,665 kandidaten in 0.072s
Wheel-30 Py:  266,665 kandidaten in 0.119s
Wheel-30 NB:  266,665 kandidaten in 0.001s
Speedup Numba: 161.3×


In [5]:
# Zelle 5 – Segmentiertes Sieb (reines Python + NumPy)
import numpy as np


def simple_sieve(limit: int) -> np.ndarray:
    """Einfaches Sieb für kleine Primzahlen (Basis für Segmentierung)."""
    if limit < 2:
        return np.array([], dtype=np.int32)
    is_prime = np.ones(limit, dtype=bool)
    is_prime[:2] = False
    for i in range(2, int(limit**0.5) + 1):
        if is_prime[i]:
            is_prime[i * i : limit : i] = False
    return np.nonzero(is_prime)[0].astype(np.int32)


def sieve_segment(start: int, end: int, base_primes: np.ndarray) -> np.ndarray:
    """Siebt Segment [start, end) mit gegebenen Basis-Primzahlen."""
    size = end - start
    if size <= 0:
        return np.array([], dtype=np.int32)
    is_prime = np.ones(size, dtype=bool)
    for p in base_primes:
        if p * p > end:
            break
        first = ((start + p - 1) // p) * p
        first = max(first, p * p)
        for j in range(first - start, size, p):
            is_prime[j] = False
    result = np.nonzero(is_prime)[0] + start
    return result.astype(np.int32)


# Test der Basis-Funktionen
base_primes = simple_sieve(100)
print(f"Basis-Primzahlen bis 100: {len(base_primes)} Stück")
segment_primes = sieve_segment(1000, 1100, base_primes)
print(f"Primzahlen in [1000, 1100): {segment_primes}")

Basis-Primzahlen bis 100: 25 Stück
Primzahlen in [1000, 1100): [1009 1013 1019 1021 1031 1033 1039 1049 1051 1061 1063 1069 1087 1091
 1093 1097]


In [6]:
# Zelle 6 – Wheel-30 + Segmentiertes Sieb (reines Python)
# Zelle 6 – Wheel-30 + Segmentiertes Sieb (reines Python, korrigiert)
import numpy as np


def sieve_segment_candidates_py(
    candidates: np.ndarray, base_primes: np.ndarray
) -> np.ndarray:
    """
    Siebt eine Liste von Kandidaten mit Basis-Primzahlen (reines Python).
    """
    mask = np.ones(len(candidates), dtype=bool)
    for p in base_primes:
        if p <= 5:  # 2,3,5 bereits eliminiert
            continue
        if p * p > candidates[-1]:
            break
        # Markiere Vielfache
        mod = candidates % p
        mask &= mod != 0
    return candidates[mask]


def wheel30_segmented_sieve(limit: int, segment_size: int = 32768) -> np.ndarray:
    if limit <= 2:
        return np.array([], dtype=np.int32)

    sqrt_limit = int(limit**0.5) + 1
    base_primes = simple_sieve(sqrt_limit)
    all_primes = []

    current = 2
    while current < limit:
        end = min(current + segment_size, limit)

        # Wheel-30-Kandidaten erzeugen
        segment_candidates = []
        k = current // WHEEL30_SIZE
        while True:
            base = k * WHEEL30_SIZE
            if base >= end:
                break
            for r in WHEEL30_COPRIME:
                candidate = base + r
                if candidate >= end:
                    break
                if candidate >= current:
                    segment_candidates.append(candidate)
            k += 1

        if segment_candidates:
            arr = np.array(segment_candidates, dtype=np.int32)
            seg_primes = sieve_segment_candidates_py(arr, base_primes)
            all_primes.extend(seg_primes.tolist())

        current = end

    return np.array(all_primes, dtype=np.int32)


# Test Korrektheit
test_primes = wheel30_segmented_sieve(100000)
print(f"Primzahlen bis 100.000: {len(test_primes)} (erwartet 9592)")

Primzahlen bis 100.000: 9550 (erwartet 9592)


In [7]:
def standard_sieve_python(limit: int) -> List[int]:
    """Standard-Sieb des Eratosthenes (Python-Referenz)."""
    if limit < 2:
        return []
    is_prime = [True] * limit
    is_prime[0] = is_prime[1] = False

    for i in range(2, int(limit**0.5) + 1):
        if is_prime[i]:
            for j in range(i * i, limit, i):
                is_prime[j] = False

    return [i for i in range(2, limit) if is_prime[i]]


SIEVE_LIMIT = 1_000_000

# Standard-Sieb (Python)
start = time.perf_counter()
standard_primes = standard_sieve_python(SIEVE_LIMIT)
standard_time = time.perf_counter() - start

# Einfaches Numba-Sieb
start = time.perf_counter()
simple_primes = simple_sieve(SIEVE_LIMIT)
simple_time = time.perf_counter() - start

# Wheel-30 Segmentiertes Sieb
start = time.perf_counter()
wheel_seg_primes = wheel30_segmented_sieve(SIEVE_LIMIT)
wheel_seg_time = time.perf_counter() - start

print(f"Primzahl-Siebe bis {SIEVE_LIMIT:,}:")
print(f"Standard Python:     {len(standard_primes):,} primes in {standard_time:.3f}s")
print(f"Simple Numba:        {len(simple_primes):,} primes in {simple_time:.3f}s")
print(f"Wheel-30 Segmented:  {len(wheel_seg_primes):,} primes in {wheel_seg_time:.3f}s")
print(f"Speedup vs Python:   {standard_time/wheel_seg_time:.1f}×")
print(f"Speedup vs Numba:    {simple_time/wheel_seg_time:.1f}×")

# Verifikation
print(
    f"Alle Methoden identisch: {len(standard_primes) == len(simple_primes) == len(wheel_seg_primes)}"
)

Primzahl-Siebe bis 1,000,000:
Standard Python:     78,498 primes in 0.302s
Simple Numba:        78,498 primes in 0.007s
Wheel-30 Segmented:  78,456 primes in 0.363s
Speedup vs Python:   0.8×
Speedup vs Numba:    0.0×
Alle Methoden identisch: False


In [8]:
import wheel_sieve
import time

# Test simple_sieve
primes10 = wheel_sieve.simple_sieve(100)
print("Erste 10 Primzahlen bis 100:", primes10[:10])

# Test wheel30_segmented_sieve und Benchmark
start = time.perf_counter()
seg_primes = wheel_sieve.wheel30_segmented_sieve(1_000_000)
elapsed = time.perf_counter() - start
print(f"Gefundene Primzahlen <1e6: {len(seg_primes)} in {elapsed:.3f}s")


Erste 10 Primzahlen bis 100: [ 2  3  5  7 11 13 17 19 23 29]
Gefundene Primzahlen <1e6: 78754 in 0.655s


In [9]:
import time
from modul2_wheel_sieve_numba import simple_sieve, wheel30_segmented_sieve

print("Erste 10 Primzahlen bis 100:", simple_sieve(100)[:10])
start = time.perf_counter()
primes = wheel30_segmented_sieve(1_000_000)
elapsed = time.perf_counter() - start
print(f"Primzahlen <1e6: {primes.size} in {elapsed:.3f}s")

Erste 10 Primzahlen bis 100: [ 2  3  5  7 11 13 17 19 23 29]
Primzahlen <1e6: 78459 in 1.352s


In [19]:
import time
from modul2_simple_sieve_numba import simple_sieve

# Test Korrektheit
primes10 = simple_sieve(100)
print("Erste 10 Primzahlen bis 100:", primes10[:10])

# Benchmark bis 1 000 000
start = time.perf_counter()
primes = simple_sieve(1_000_000)
elapsed = time.perf_counter() - start
print(f"Primzahlen <1e6: {len(primes):,} in {elapsed:.3f}s")

Erste 10 Primzahlen bis 100: [ 2  3  5  7 11 13 17 19 23 29]
Primzahlen <1e6: 78,498 in 0.007s
